In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col,lit
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

df = spark.read.option("header", "true").csv("/home/iceberg/data/events.csv").withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))

df.show()
# df.join(df, lit(1)==lit(1)).take(5)

24/12/05 16:05:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----------+----------+--------+--------------------+----------+--------------------+-------------------+
|    user_id| device_id|referrer|                host|       url|          event_time|         event_date|
+-----------+----------+--------+--------------------+----------+--------------------+-------------------+
| 1037710827| 532630305|    NULL| www.zachwilson.tech|         /|2021-03-08 17:27:...|2021-03-08 00:00:00|
|  925588856| 532630305|    NULL|    www.eczachly.com|         /|2021-05-10 11:26:...|2021-05-10 00:00:00|
|-1180485268| 532630305|    NULL|admin.zachwilson....|         /|2021-02-17 16:19:...|2021-02-17 00:00:00|
|-1044833855| 532630305|    NULL| www.zachwilson.tech|         /|2021-09-24 15:53:...|2021-09-24 00:00:00|
|  747494706| 532630305|    NULL| www.zachwilson.tech|         /|2021-09-26 16:03:...|2021-09-26 00:00:00|
|  747494706| 532630305|    NULL|admin.zachwilson....|         /|2021-02-21 16:08:...|2021-02-21 00:00:00|
| -824540328| 532630305|    NULL|admi

In [2]:
sorted = df.repartition(100, col("event_date")) \
        .sortWithinPartitions(col("event_date"), col("host")) \
           .withColumn("event_time", col("event_time").cast("timestamp")) \

sorted.show()
 # col("browser_family")

+-----------+-----------+--------------------+--------------------+-----------+--------------------+-------------------+
|    user_id|  device_id|            referrer|                host|        url|          event_time|         event_date|
+-----------+-----------+--------------------+--------------------+-----------+--------------------+-------------------+
|  -94037775|-2012543895|                NULL|admin.zachwilson....|/robots.txt|2021-05-30 12:29:...|2021-05-30 00:00:00|
|  760642255|-1883465659|                NULL|admin.zachwilson....|          /|2021-05-30 12:29:...|2021-05-30 00:00:00|
| 1587120550| -290659081|                NULL|    www.eczachly.com|          /|2021-05-30 00:00:...|2021-05-30 00:00:00|
| -664256598| -290659081|                NULL|    www.eczachly.com|      /blog|2021-05-30 01:48:...|2021-05-30 00:00:00|
| 1896286514|  532630305|                NULL|    www.eczachly.com|          /|2021-05-30 03:31:...|2021-05-30 00:00:00|
| 1896286514|  532630305|       

In [4]:
sortedTwo = df.repartition(100, col("event_date")) \
        .sort(col("event_date"), col("host")) \
           .withColumn("event_time", col("event_time").cast("timestamp")) 
sortedTwo.show()

+-----------+-----------+--------+--------------------+--------------------+--------------------+-------------------+
|    user_id|  device_id|referrer|                host|                 url|          event_time|         event_date|
+-----------+-----------+--------+--------------------+--------------------+--------------------+-------------------+
| 1272828233| -643696601|    NULL|admin.zachwilson....|                   /|2021-01-02 13:53:...|2021-01-02 00:00:00|
|  747494706|  532630305|    NULL|admin.zachwilson....|                   /|2021-01-02 19:36:...|2021-01-02 00:00:00|
| 2110046626|  898871897|    NULL|admin.zachwilson....|       /wp-login.php|2021-01-02 19:57:...|2021-01-02 00:00:00|
| 1272828233| -643696601|    NULL|admin.zachwilson....|                   /|2021-01-02 21:05:...|2021-01-02 00:00:00|
| 1272828233| -643696601|    NULL|admin.zachwilson....|                   /|2021-01-02 21:37:...|2021-01-02 00:00:00|
| 1399665425|-2012543895|    NULL|    www.eczachly.com| 

In [5]:
sorted.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, cast(event_time#22 as timestamp) AS event_time#73, event_date#29]
   +- Sort [event_date#29 ASC NULLS FIRST, host#20 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(event_date#29, 100), REPARTITION_BY_NUM, [plan_id=132]
         +- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, event_time#22, date_trunc(day, cast(event_time#22 as timestamp), Some(Etc/UTC)) AS event_date#29]
            +- FileScan csv [user_id#17,device_id#18,referrer#19,host#20,url#21,event_time#22] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/iceberg/data/events.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user_id:string,device_id:string,referrer:string,host:string,url:string,event_time:string>




In [6]:
sortedTwo.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, cast(event_time#22 as timestamp) AS event_time#116, event_date#29]
   +- Sort [event_date#29 ASC NULLS FIRST, host#20 ASC NULLS FIRST], true, 0
      +- Exchange rangepartitioning(event_date#29 ASC NULLS FIRST, host#20 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=154]
         +- Exchange hashpartitioning(event_date#29, 100), REPARTITION_BY_NUM, [plan_id=150]
            +- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, event_time#22, date_trunc(day, cast(event_time#22 as timestamp), Some(Etc/UTC)) AS event_date#29]
               +- FileScan csv [user_id#17,device_id#18,referrer#19,host#20,url#21,event_time#22] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/iceberg/data/events.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user_id:string,device_id:string,referrer:string,ho

in the .explain:
Project and select mean the same thing. </br>
Exchange hashpartitioning does the partitioning. </br>
In the sort line, there is a boolean parameter which tells us if the sort is global or not. </br>
The Exchange rangepartitioning is the painful part (at scale). </br>
Exchange means shuffle!

In [2]:
%%sql

CREATE DATABASE IF NOT EXISTS bootcamp

++
||
++
++

In [4]:
%%sql 
SHOW TABLES IN bootcamp;

namespace,tableName,isTemporary
bootcamp,events,False
bootcamp,events_sorted,False
bootcamp,events_unsorted,False


In [9]:
%%sql

DROP TABLE IF EXISTS iceberg.bootcamp.events

++
||
++
++

In [11]:
-- %%sql

-- CREATE TABLE IF NOT EXISTS bootcamp.events (
--     url STRING,
--     referrer STRING,
--     browser_family STRING,
--     os_family STRING,
--     device_family STRING,
--     host STRING,
--     event_time TIMESTAMP,
--     event_date DATE
-- )
-- USING iceberg
--     PARTITIONED BY (event_date);
-- PARTITIONED BY (years(event_date));


++
||
++
++

In [14]:
%%sql

CREATE TABLE IF NOT EXISTS iceberg.bootcamp.events (
    user_id NUMERIC,
    device_id NUMERIC,
    referrer STRING,
    host STRING,
    url STRING,

    event_time TIMESTAMP,
    event_date DATE
    --browser_family STRING,
    --os_family STRING,
    --device_family STRING,
)
USING iceberg
    PARTITIONED BY (event_date);

++
||
++
++

In [15]:
%%sql


CREATE TABLE IF NOT EXISTS iceberg.bootcamp.events_sorted (
    user_id NUMERIC,
    device_id NUMERIC,
    referrer STRING,
    host STRING,
    url STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
    PARTITIONED BY (event_date);
    -- url STRING,
    -- referrer STRING,
    -- browser_family STRING,
    -- os_family STRING,
    -- device_family STRING,
    -- host STRING,
    -- event_time TIMESTAMP,
    -- event_date DATE

++
||
++
++

In [16]:
%%sql


CREATE TABLE IF NOT EXISTS iceberg.bootcamp.events_unsorted (
    user_id NUMERIC,
    device_id NUMERIC,
    referrer STRING,
    host STRING,
    url STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
    PARTITIONED BY (event_date);

++
||
++
++

In [21]:

start_df = df.repartition(4, col("event_date")).withColumn("event_time", col("event_time").cast("timestamp")) \
    

first_sort_df = start_df.sortWithinPartitions(col("event_date"), col("host")) #col("browser_family"),

# sorted = df.repartition(10, col("event_date")) \
#         .sortWithinPartitions(col("event_date")) \
#         .withColumn("event_time", col("event_time").cast("timestamp")) \

start_df.write.mode("overwrite").saveAsTable("iceberg.bootcamp.events_unsorted")
first_sort_df.write.mode("overwrite").saveAsTable("iceberg.bootcamp.events_sorted")

ERROR:root:Exception while sending command.                         (0 + 4) / 4]
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: An error occurred while calling o219.saveAsTable

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [22]:
# Check a specific configuration
warehouse_path = spark.conf.get("spark.sql.catalog.iceberg.warehouse", "Not Set")
print("Warehouse Path:", warehouse_path)

ConnectionRefusedError: [Errno 111] Connection refused

In [121]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM demo.bootcamp.events_sorted.files

UNION ALL
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM demo.bootcamp.events_unsorted.files





size,num_files,sorted
2896920,4,sorted
3211534,4,unsorted


In [90]:
%%sql
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files FROM demo.bootcamp.events.files;

size,num_files
3145713,5


In [ ]:
%%sql 
SELECT COUNT(1) FROM bootcamp.matches_bucketed.files

count(1)
3665
